In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
tqdm.pandas()

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/PCN_Project/클러스터링colab/Final_fire_news_4.csv').drop_duplicates()
df

In [ ]:
def cleansing(body, mode=None):
    if type(body) == np.ndarray:
        body = np.round(body.tolist(), 5)
        body = body.tolist()
        return body
    else:
        body = re.sub('<YNAOBJECT.*?/YNAOBJECT>', '', body, 0, re.I|re.S) # YNAOBJECT 태그 제거
        body = re.sub('<table.*?/table>', '', body, 0, re.I|re.S) # table 태그 제거
        
        if mode == 'meta':
            try:
                body = body[re.search("\(.*?연합뉴스\).*?=", body).span()[0]:]
            except AttributeError:
                pass
            return body
        
        if mode == 'sum':
            try:
                body = body[re.search("\(.*?연합뉴스\).*?=", body).span()[1]:]
            except AttributeError:
                pass

        body = re.sub('[""]', '', body, 0, re.I|re.S) # "" 제거
        body = re.sub("['']", '', body, 0, re.I|re.S) # '' 제거
        body = re.sub(",", '', body, 0, re.I|re.S) # , 제거
#         body = re.sub(r'[^ A-Za-z가-힣+]',' ' , body, 0, re.I|re.S) # 한글, 알파벳을 제외한 나머지 제거

        paragraphs = body.split('\r\n') # 단락 분리
        if ' 기자' in paragraphs[-1]: del paragraphs[-1] # 끝 단락 기자명 제거
        body = []
        for paragraph in paragraphs:
            paragraph = re.sub('\[.+?\]', '', paragraph, 0, re.I|re.S)
            paragraph = ' '.join(paragraph.split()) # 문자열 중간 다중 공백 제거
            if len(paragraph) > 0:
                body.append(paragraph)
        
        if mode == 'quot':
            return body
        else:
            return ' '.join(body)

In [ ]:
df['clean_sum'] = df['Summarization'].progress_apply(cleansing)
df['clean_sum']

100%|██████████| 14057/14057 [00:00<00:00, 37088.45it/s]


0         
1         
2         
3         
10        
        ..
35910     
35911     
35912     
35913     
35914     
Name: clean_sum, Length: 14057, dtype: object

In [ ]:
df['class_name'].value_counts()

안전관리    3662
풍수해     1822
사고일반    1587
태풍       994
화재       898
육상사고     820
안전사고     610
재해일반     509
차사고      505
산재       378
음주사고     261
지진       225
수상사고     219
가뭄       174
철도사고     157
폭설       139
항공사고     137
폭발사고      87
핵사고       22
Name: class_name, dtype: int64

In [ ]:
안전관리 = df[df.class_name == '안전관리']

In [ ]:
안전관리.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3662 entries, 47 to 35914
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   _id             3662 non-null   object 
 1   send_timestamp  3662 non-null   object 
 2   Title           3662 non-null   object 
 3   Body            3662 non-null   object 
 4   Summarization   3662 non-null   object 
 5   NamedEntity     3662 non-null   object 
 6   Keyword         3662 non-null   object 
 7   img_link        3631 non-null   object 
 8   issue_word      939 non-null    object 
 9   reason          34 non-null     object 
 10  class_code      3662 non-null   float64
 11  class_name      3662 non-null   object 
 12  clean_sum       3662 non-null   object 
dtypes: float64(1), object(12)
memory usage: 400.5+ KB


## Clustering

### DBSCAN Clustering

- 밀도 차이 기반 알고리즘
- K-means clusterng과 달리 군집의 개수 k를 지정해줄 필요가 없으며, 알고리즘이 자체적으로 데이터 밀도 차이를 감지하여 군집을 생성
- 데이터 밀도가 자주 변하거나, 밀도 차이가 극명하지 않은 데이터에는 좋지않음
- 문맥을 고려하기 위해 TF-IDF Vectorization을 진행
- 중복 기사들끼리만 묶기 위해서, 군집화 기준을 까다롭게 설정하였다. 
    - epsilon 값은 0.1로 설정하여 군집 내부 유사도는 매우 높게 설정
    - min_samples 값을 1로 설정하여 군집 간 변별력을 낮춤

In [ ]:
#1 tf-idf 임베딩(+Normalize)

from sklearn.feature_extraction.text import TfidfVectorizer

text = 안전관리['Summarization'].tolist()

tfidf_vectorizer = TfidfVectorizer(min_df = 3, ngram_range=(1,5))
tfidf_vectorizer.fit(text)
vector = tfidf_vectorizer.transform(text).toarray()

vector = np.array(vector)

In [ ]:
#2 DBSCAN Clustering
from sklearn.cluster import DBSCAN

model = DBSCAN(eps=0.1,min_samples=1, metric = "cosine") 
#     거리 계산 식으로는 Cosine distance를 이용
#     eps이 낮을수록, min_samples 값이 높을수록 군집으로 판단하는 기준이 까다로움.
result = model.fit_predict(vector)
안전관리['cluster1st'] = result

print('군집개수 :', result.max())
안전관리.columns

군집개수 : 3248


<ipython-input-10-622fe8caf99c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  안전관리['cluster1st'] = result


Index(['_id', 'send_timestamp', 'Title', 'Body', 'Summarization',
       'NamedEntity', 'Keyword', 'img_link', 'issue_word', 'reason',
       'class_code', 'class_name', 'clean_sum', 'cluster1st'],
      dtype='object')

In [ ]:
#3 대표 기사 추출
def print_cluster_result(train):
    clusters = []
    counts = []
    top_title = []
    top_noun = []
    for cluster_num in set(result):
        # -1,0은 노이즈 판별이 났거나 클러스터링이 안된 경우
        # if(cluster_num == -1 or cluster_num == 0): 
        #     continue
        # else:
            # print("cluster num : {}".format(cluster_num))
            temp_df = train[train['cluster1st'] == cluster_num] # cluster num 별로 조회
            clusters.append(cluster_num)
            counts.append(len(temp_df))
            top_title.append(temp_df.reset_index()['Title'][0])
            top_noun.append(temp_df.reset_index()['Summarization'][0]) # 군집별 첫번째 기사를 대표기사로 ; tfidf방식
            # for title in temp_df['YNewsML.NewsContent.Title']:
            #     print(title) # 제목으로 살펴보자
            # print()
    cluster_result = pd.DataFrame({'cluster_num':clusters, 'count':counts, 'top_title':top_title, 'top_noun':top_noun})
    return cluster_result

cluster1_result = print_cluster_result(안전관리)
cluster1_result

,cluster_num,count,top_title,top_noun
0,0,1,탈선 후 1년째 휴장…애물단지 된 통영 욕지도 모노레일,['지난해 11월 탈선 사고로 운행을 멈춘 경남 통영시 욕지도 모노레일이 1년 넘게...
1,1,1,"[신년사] 오세훈 ""안전에 모든 노력…동행·매력도시 도약""","['오세훈 서울시장은 2023년 새해를 맞아 ""서울시민이 언제 어디를 가더라도 안전..."
2,2,1,충북 3년만에 열리는 해맞이 행사…안전대책 총력,"['새해 첫날 충북 곳곳에서 계묘년 해맞이 행사와 공연이 펼쳐진다.', '도민 1천..."
3,3,1,대전·세종·충남 국도 내 방음터널 8곳…세종시 긴급 안전 점검,['5명의 목숨을 앗아간 경기 과천 제2경인고속도로 방음터널 화재와 관련해 방음터널...
4,4,1,서울 방음터널 16곳 중 4곳 '화재취약' 아크릴 소재,['서울 시내 방음터널 총 16곳 중 4곳은 화재에 취약한 폴리메타크릴산메틸(PMM...
...,...,...,...,...
3244,3244,1,"한국소비자원, 편의점 업계와 여름철 먹거리 안전 관리 캠페인",['한국소비자원은 여름철을 맞아 편의점 사업자 정례협의체와 함께 먹거리 안전을 위한...
3245,3245,1,"소방청-가스안전공사, 신에너지 사고 대응 업무협약",['소방청은 8일 한국가스안전공사와 수소 등 신에너지 관련 사고에 대응하기 위해 업...
3246,3246,1,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축","[""LGU+, 서울지하철 8호선에 '스마트 역사' 구축 LG유플러스가 서울 지하철 ..."
3247,3247,1,"LGU+, 서울지하철 8호선에 '스마트 역사' 구축 완료","[""LG유플러스는 서울 지하철 8호선 18개 역사에 '스마트스테이션'을 구축하는 사..."


### K-Means Clustering
- k-Means Clustering은 사전에 군집의 개수 k를 설정해주어야 한다. 
- 앞서 DBSCAN을 이용해 구축한 311개의 뉴스 기사 데이터를 다시 TF-IDF Vectorization 한 후, K-Means Clustering 기법을 이용
- k개의 군집은 각 하나의 중심점을 가지고, 각 데이터는 가까운 중심점에 할당된다.
- 각 군집과 데이터 간의 거리 분산을 최소화하는 방식으로 클러스터링이 진행된다.
- 최적의 군집 개수를 찾기 위해 두가지 방법을 고려하였다.

- Elbow Method
  - 첫 번째로는 클러스터 내의 총 변동을 설명하는 WCSS(Within Clusters Sum of Squares)를 이용하는 Elbow Method이다. 
  - 각 클러스터를 WCSS 방법으로 계산한 뒤, SSE가 가장 급격하게 줄어드는 구간에서 군집 개수 k를 결정하는 방법이다.
  - 하지만 거의 일정한 SSE 감소율로 인해 적합한 k를 결정하기에는 어려웠다.

- Silhouette Score
  - 두 번째로 고려한 방법은 다른 클러스터(seperation)에 비해 자신의 클러스터(cohesion)와의 유사도를 측도로 하는 Silhouette Score
  - 값이 높으면 객체가 자체 클러스터와 잘 일치하고 인접 클러스터와 잘 일치하지 않음을 나타낸다.
  - 실루엣 계수가 가장 높은 k=29이 최적 군집 개수라고 판단
  - 따라서 cluster 0부터 cluster 28까지, 총 29개의 군집으로 311개의 기사를 분류

In [ ]:
#1 tf-idf 임베딩(+Normalize)

from sklearn.feature_extraction.text import TfidfVectorizer

text = cluster1_result['top_noun'].tolist()

tfidf_vectorizer = TfidfVectorizer(min_df = 3, ngram_range=(1,5))
tfidf_vectorizer.fit(text)
vector_2nd = tfidf_vectorizer.transform(text).toarray()

vector_2nd = np.array(vector_2nd)

In [ ]:
# 2-1. Elbow Method
# ..
# 2-2. Silhouette Score - 최적 k
from sklearn.metrics import silhouette_samples, silhouette_scoref
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt

def visualize_silhouette_layer(data, param_init='random', param_n_init=10, param_max_iter=300):
    clusters_range = range(50,100)
    results = []

    for i in clusters_range:
      # 군집화 수행
        clusterer = KMeans(n_clusters=i, init=param_init, n_init=param_n_init, max_iter=param_max_iter, random_state=0)
        cluster_labels = clusterer.fit_predict(data)
        silhouette_avg = silhouette_score(data, cluster_labels)
        results.append([i, silhouette_avg])


    result = pd.DataFrame(results, columns=["n_clusters", "silhouette_score"])
    pivot_km = pd.pivot_table(result, index="n_clusters", values="silhouette_score")

    plt.figure()
    sns.heatmap(pivot_km, annot=True, linewidths=.5, fmt='.3f', cmap=sns.cm._rocket_lut)
    plt.tight_layout()
    plt.show()

visualize_silhouette_layer(vector_2nd) # 가장 높은 실루엣 계수와 매핑되는 k = 29

In [ ]:
#3 K-Means Clustering

from sklearn.cluster import KMeans

result_2nd = KMeans(n_clusters=25).fit_predict(vector_2nd)
cluster1_result['cluster2nd'] = result_2nd

cluster1_result

,cluster_num,count,top_title,top_noun,cluster2nd
0,0,1,과천 제2경인고속도 방음터널서 큰 불…5명 사망·37명 부상(종합4보),과천 제2경인고속도 방음터널서 큰 불5명 사망 37명 부상 사망자들 모두 주변 지나...,1
1,1,1,과천 제2경인고속도 방음터널서 화재…5명 사망·37명 부상(종합3보),과천 제2경인고속도 방음터널서 화재5명 사망 37명 부상 버스 트럭 추돌사고로 발생...,1
2,2,1,지하철 3호선 화재로 운행중단…한파속 출근길 대란(종합3보),지하철 3호선 화재로 운행중단한파속 출근길 대란 약수∼구파발 양방향 1시간45분 멈...,6
3,3,1,"특수본, 박희영 구청장·류미진 총경 피의자 소환(종합3보)",특수본 박희영 구청장 류미진 총경 피의자 소환 이상민 장관 고발사건 공수처 통보경찰...,20
4,4,1,'이태원 참사' 경찰 지휘부 정조준…조만간 줄소환(종합3보),이태원 참사 경찰 지휘부 정조준조만간 줄소환 55곳서 휴대전화 등 총 1만3천125...,9
5,5,1,봉화 광산사고 광부들 '기적의 생환'…221시간 만에 걸어나왔다(종합3보),봉화 광산사고 광부들 기적의 생환221시간 만에 걸어나왔다 소방 당국 고립자 2명 ...,4
6,6,1,핼러윈의 비극…이태원 '압사 참사' 153명 사망(종합3보),핼러윈의 비극이태원 압사 참사 153명 사망 사망자 중 여성 97명 외국인 20명세...,24
7,7,1,[이태원 참사] 실종신고 3천건 넘어…서울광장에 합동분향소(종합3보),실종신고 3천건 넘어서울광장에 합동분향소 서울시 내일부터 분향소 운영화장시설 가동횟...,0
8,8,3,이태원 '핼러윈 인파'에 149명 압사 참사…부상 76명(종합5보),이태원 핼러윈 인파에 149명 압사 참사부상 76명 중상 19명 사망자 더 늘어날 ...,3
9,9,1,"이태원 '핼러윈의 악몽'…""도미노처럼 넘어지며 5∼6겹 쌓여""(종합3보)",이태원 핼러윈의 악몽도미노처럼 넘어지며 5∼6겹 쌓여 인파 한꺼번에 몰리며 순식간에...,17


In [ ]:
#4 대표 기사 추출
def print_cluster_result(train):
    clusters = []
    counts = []
    top_title = []
    top_noun = []
    for cluster_num in set(result_2nd):
        # -1,0은 노이즈 판별이 났거나 클러스터링이 안된 경우
        # if(cluster_num == -1 or cluster_num == 0): 
        #     continue
        # else:
            # print("cluster num : {}".format(cluster_num))
            temp_df = train[train['cluster2nd'] == cluster_num] # cluster num 별로 조회
            clusters.append(cluster_num)
            counts.append(len(temp_df))
            top_title.append(temp_df.reset_index()['top_title'][0])
            top_noun.append(temp_df.reset_index()['top_noun'][0]) # 군집별 첫번째 기사를 대표기사로 ; tfidf방식
            # for title in temp_df['YNewsML.NewsContent.Title']:
            #     print(title) # 제목으로 살펴보자
            # print()
    cluster_result = pd.DataFrame({'cluster_num':clusters, 'count':counts, 'top_title':top_title, 'top_noun':top_noun})
    return cluster_result

cluster2_result = print_cluster_result(train=cluster1_result)
cluster2_result

,cluster_num,count,top_title,top_noun
0,0,1,[이태원 참사] 실종신고 3천건 넘어…서울광장에 합동분향소(종합3보),실종신고 3천건 넘어서울광장에 합동분향소 서울시 내일부터 분향소 운영화장시설 가동횟...
1,1,2,과천 제2경인고속도 방음터널서 큰 불…5명 사망·37명 부상(종합4보),과천 제2경인고속도 방음터널서 큰 불5명 사망 37명 부상 사망자들 모두 주변 지나...
2,2,1,[집중호우] 여의도 면적 3배 농작물 침수피해…절반은 충남(종합3보),여의도 면적 3배 농작물 침수피해절반은 충남 사망 12명 실종 7명이재민 1천490...
3,3,1,이태원 '핼러윈 인파'에 149명 압사 참사…부상 76명(종합5보),이태원 핼러윈 인파에 149명 압사 참사부상 76명 중상 19명 사망자 더 늘어날 ...
4,4,1,봉화 광산사고 광부들 '기적의 생환'…221시간 만에 걸어나왔다(종합3보),봉화 광산사고 광부들 기적의 생환221시간 만에 걸어나왔다 소방 당국 고립자 2명 ...
5,5,1,"SPC계열 샤니 제빵공장서 손 끼임 사고…""안전 점검 중""(종합3보)",SPC계열 샤니 제빵공장서 손 끼임 사고안전 점검 중 SPC 빵 상자 검수 작업 중...
6,6,1,지하철 3호선 화재로 운행중단…한파속 출근길 대란(종합3보),지하철 3호선 화재로 운행중단한파속 출근길 대란 약수∼구파발 양방향 1시간45분 멈...
7,7,1,종로 르메이에르빌딩 5분간 흔들…1천여명 한때 대피(종합3보),종로 르메이에르빌딩 5분간 흔들1천여명 한때 대피 옥상 냉각타워 파손 영향날개 부러...
8,8,1,[태풍 힌남노] 일본 나가사키현 초속 32.9ｍ 강풍(종합3보),일본 나가사키현 초속 32.9ｍ 강풍 미야자키현 시간당 51㎜ 집중호우힌남노 규슈 ...
9,9,1,'이태원 참사' 경찰 지휘부 정조준…조만간 줄소환(종합3보),이태원 참사 경찰 지휘부 정조준조만간 줄소환 55곳서 휴대전화 등 총 1만3천125...


# 키워드 추출


### KeyBERT
- 군집 내의 모든 타이틀을 하나의 텍스트로 이어 KeyBERT 모델에 넣었고, '전체 문장'과 가장 유사한 키워드를 추출
- 워드임베딩 방식을 포함해서 다양한 임베딩 모델을 지원했는데, sentence-tansformer 방식 중, paraphrase-multilingual-MiniLM-L12-v2 모델을 사용
- Manual Search로 keyphrase_ngram_range, use_mmr, diversity, use_maxsum, nr_candidate 등의 하이퍼파라미터를 조정
- 워드 추출 결과를 확인해본 결과, 인접한 하나나 두 개의 단어로 이루어진 후보군들 중에서 코사인 유사도를 기반으로 키워드를 추출하는 방식이 가장 좋은 성능을 보였다.

In [ ]:
!pip install keybert

In [ ]:
from keybert import KeyBERT

key_model = KeyBERT('paraphrase-multilingual-MiniLM-L12-v2')  #distilbert-base-nli-mean-tokens / paraphrase-multilingual-MiniLM-L12-v2

In [ ]:
def keyword(data, col_cluster):  #data = cluster_result (데이터프레임) #1분 30초 소요됨
    result = []
    for i in range(len(data)):
        key_text = cluster1_result[cluster1_result[col_cluster]==i]['top_title']
        key_text = ' '.join(key_text)
        keyword = key_model.extract_keywords(key_text, keyphrase_ngram_range=(1,2), top_n=1)
        result.append(keyword[0][0])
    return result

def merge_keyword(data, col_cluster): #새 열로 추가.
    data_temp = data.copy()
    data_temp['keyword'] = keyword(data, col_cluster)
    return data_temp

keyword_result = merge_keyword(cluster2_result, col_cluster='cluster2nd')

keyword_df = keyword_result[['cluster_num', 'count', 'keyword']]
keyword_df.sort_values(by='count', ascending=False, inplace=True, ignore_index=True)
# keyword_df.drop(index=[0], inplace=True)
# keyword_df = keyword_df[keyword_df['count']>5]
lst = []
for i in keyword_df['keyword']:
  lst.append(i.upper()) 
keyword_df['keyword'] = lst
keyword_df.sort_values('cluster_num')

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
<ipython-input-32-a94f2eb1f09a>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  keyword_df['keyword'] = lst


,cluster_num,count,keyword
1,0,1,참사 실종신고
0,1,2,방음터널서 화재
23,2,1,집중호우 여의도
22,3,1,이태원 핼러윈
21,4,1,걸어나왔다 종합3보
20,5,1,안전 점검
19,6,1,화재로
18,7,1,종로 르메이에르빌딩
17,8,1,나가사키현 초속
16,9,1,지휘부 정조준


In [ ]:
merge_outer = pd.merge(cluster2_result,keyword_df, how='left',on='cluster_num')
merge_outer

,cluster_num,count_x,top_title,top_noun,count_y,keyword
0,0,1,[이태원 참사] 실종신고 3천건 넘어…서울광장에 합동분향소(종합3보),실종신고 3천건 넘어서울광장에 합동분향소 서울시 내일부터 분향소 운영화장시설 가동횟...,1,참사 실종신고
1,1,2,과천 제2경인고속도 방음터널서 큰 불…5명 사망·37명 부상(종합4보),과천 제2경인고속도 방음터널서 큰 불5명 사망 37명 부상 사망자들 모두 주변 지나...,2,방음터널서 화재
2,2,1,[집중호우] 여의도 면적 3배 농작물 침수피해…절반은 충남(종합3보),여의도 면적 3배 농작물 침수피해절반은 충남 사망 12명 실종 7명이재민 1천490...,1,집중호우 여의도
3,3,1,이태원 '핼러윈 인파'에 149명 압사 참사…부상 76명(종합5보),이태원 핼러윈 인파에 149명 압사 참사부상 76명 중상 19명 사망자 더 늘어날 ...,1,이태원 핼러윈
4,4,1,봉화 광산사고 광부들 '기적의 생환'…221시간 만에 걸어나왔다(종합3보),봉화 광산사고 광부들 기적의 생환221시간 만에 걸어나왔다 소방 당국 고립자 2명 ...,1,걸어나왔다 종합3보
5,5,1,"SPC계열 샤니 제빵공장서 손 끼임 사고…""안전 점검 중""(종합3보)",SPC계열 샤니 제빵공장서 손 끼임 사고안전 점검 중 SPC 빵 상자 검수 작업 중...,1,안전 점검
6,6,1,지하철 3호선 화재로 운행중단…한파속 출근길 대란(종합3보),지하철 3호선 화재로 운행중단한파속 출근길 대란 약수∼구파발 양방향 1시간45분 멈...,1,화재로
7,7,1,종로 르메이에르빌딩 5분간 흔들…1천여명 한때 대피(종합3보),종로 르메이에르빌딩 5분간 흔들1천여명 한때 대피 옥상 냉각타워 파손 영향날개 부러...,1,종로 르메이에르빌딩
8,8,1,[태풍 힌남노] 일본 나가사키현 초속 32.9ｍ 강풍(종합3보),일본 나가사키현 초속 32.9ｍ 강풍 미야자키현 시간당 51㎜ 집중호우힌남노 규슈 ...,1,나가사키현 초속
9,9,1,'이태원 참사' 경찰 지휘부 정조준…조만간 줄소환(종합3보),이태원 참사 경찰 지휘부 정조준조만간 줄소환 55곳서 휴대전화 등 총 1만3천125...,1,지휘부 정조준
